In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import torch

- https://arxiv.org/abs/1811.12560
- https://pathmind.com/wiki/deep-reinforcement-learning    
- https://towardsdatascience.com/why-going-from-implementing-q-learning-to-deep-q-learning-can-be-difficult-36e7ea1648af   
https://towardsdatascience.com/policy-based-reinforcement-learning-the-easy-way-8de9a3356083

- https://github.com/deepmind/pysc2
- http://chris-chris.ai/2017/08/30/pysc2-tutorial1/
- https://github.com/chris-chris/pysc2-examples
- https://blog.goodaudience.com/lessons-and-mistakes-from-my-first-reinforcement-learning-starcraft-agent-4245cc35e956
- https://faculty.utrgv.edu/dongchul.kim/csci4352/spring2019/report/R5.pdf
- http://courses.cecs.anu.edu.au/courses/CSPROJECTS/19S1/reports/u6049249_report.pdf
- https://chatbotslife.com/building-a-smart-pysc2-agent-cdc269cb095d

- https://cs.stanford.edu/people/karpathy/reinforcejs/

## Deep Reinforced Learning

Deep Neural Networks's power to represent the world

The ability to act on that understanding

Deep Learning: Takes samples from data, encode the representation in a way we can reason about it

Sequential decision making problem

An inteligenet agent has to make decisions that affect the world

How do we learn anything? Starting from so little?

Using the frameworks of DL. The neural net is doign the representation of the world on which the actions are made

Value learning

Find $Q(s,t)$, then $a = argmax_a Q(s,t)$

Deep Q network (DQN)

Neural network that recieves a state and returns $Q(s, a_1), Q(s, a_2), \ldots, Q(s, a_N)$

maximize target return

target is $r$ current reward plus \gamma \max_a' Q(a',s')$

prediced is $Q(s, a)$

loss is $\mathbb{E}[\| (r + \gamma \max_a' Q(a',s') - Q(s,a)\|^2]$ called Q-loss

Q-learning, discrete and small action space, cannot handle continous action space, does not learn stochastic policies, we solve some of this with policy gradient learning

Policy learning

Find $\pi(s)$, sample $a\sim\pi(s)$

Neural network that recieves a state and returns $P(a_1|s), P(a_2|s), \ldots, P(a_N|s)$. You draw a sample from this distribution

advantages? distribution does not need to be categorical, can handle continuous, parametrize $\mu$ and $\sigma$ 


Training algorithm

- Run policy until termination, record tuples actions/states/rewards
- Decrease pbb of actions that ended on low reward 
- Increase pbb of actions that ended on high reward

$$
L = - \log P(a_t|s_t) R_t
$$

log likelihood of selecting the action, how likely was this action that you selected

total discounted returned recieved by selecting that action

- high reward times likely action -> desirable, will not change
- reward is low times likely action -> undesirable, will minimize: remove pbb of this action

$$
w = w + \mu \nabla \log P(a_t|s_t) R_t
$$

policy gradient

real life: run until termination!!!!! Requires simulator!!!!
- realistic simulation + transfer learning
- real world observations + one shot trial & error


### Agente

El agente está basado en uno o más de los siguientes componentes

- Policy: Indica como escogemos las acciones, comportamiento del agente
- Value: Se usa para evaluar la "calidad" de los estados estados
- Model: El agente mantiene una representación aproximada del ambiente 


No siempre están presentes las tres

- Agentes basados en modelo versus agentes *model-free*
- Agentes basados en policy versus agentes basados en value





#### Modelo/*Model*

Hay dos cosas que el agente busca modelar

¿Cúal será el siguiente estado?

Se modela como una probabilidad de transicionar al estado $s'$ dado que estamos en $s$ y tomamos la acción $a$

$$
\mathcal{P}_{ss'}^a = P(S_{t+1}=s'|S_t=s|A_t =a)
$$


¿Cúal será la siguiente recompensa?

Se modela como el valor esperado de recompensa dado que tomo la acción $a$ en el estado $s$

$$
\mathcal{R}_s^a = \mathbb{E}[R|S_t=s, A_t=a]
$$

Expected total future reward Q function


$$
Q(s_t, a_t) = \mathbb{E}[R_t|s_t, a_t]
$$

The policy should choose an action that maximizes future reward
$$
\pi(s)^* = argmax_a Q(s_t, a_t)
$$



https://medium.com/@curiousily/solving-an-mdp-with-q-learning-from-scratch-deep-reinforcement-learning-for-hackers-part-1-45d1d360c120

https://github.com/aamini/introtodeeplearning/blob/master/lab3/RL.ipynb